## 4D Objects By Change - Creating an analysis

This notebook explains the data preparation step for the extraction of *4D Objects-By-Change* (4D-OBCs; [Anders et al., 2020](#references)). For details about the method, we refer to the articles by Anders et al. ([2020; 2021]((#references))).

In [2]:
import py4dgeo
import numpy as np

This notebook makes use of synthetic test data. We first ensure that we have test data available:

In [3]:
py4dgeo.ensure_test_data_availability()

The main data structure for the 4D-OBC algorithm is a `SpatiotemporalAnalysis` object. It is always backed by an archive file, which we specify when instantiating the analysis object. If the given filename already exists, we open it, otherwise an empty archive is created:

In [5]:
analysis = py4dgeo.SpatiotemporalAnalysis("test.zip", force=True)

[2022-06-16 14:38:19][INFO] Creating analysis file H:\projects_hot\202206_py4dgeo\py4dgeo\jupyter\test.zip


The analysis object stores all information necessary to (re-)run and extend the analysis. This includes, e.g., the reference epoch, the core points, and the space-time array of change values. However, it does not store all epochs that were used in building up the analysis. 

In the following we show how the required components are added to the analysis using the example of the two epochs used for [demonstration of the M3C2](m3c2.ipynb). To be usable during spatiotemporal segmentation, a timestamp is added to these epochs, respectively:

In [6]:
reference_epoch, epoch = py4dgeo.read_from_xyz(
    "plane_horizontal_t1.xyz", "plane_horizontal_t2.xyz"
)
reference_epoch.timestamp = "March 9th 2022, 16:00"
epoch.timestamp = "March 10th 2022, 16:00"

[2022-06-16 14:43:19][INFO] Reading point cloud from file 'C:\Users\k53\AppData\Local\Temp\tmp0r009yki\plane_horizontal_t1.xyz'
[2022-06-16 14:43:19][INFO] Determined coordinate offset as [0.         0.         3.50018554]
[2022-06-16 14:43:19][INFO] Reading point cloud from file 'C:\Users\k53\AppData\Local\Temp\tmp0r009yki\plane_horizontal_t2.xyz'


H:\conda_envs\py4dgeo_dev\lib\site-packages\dateparser\date_parser.py:35: PytzUsageWarning: The localize method is no longer necessary, as this time zone supports the fold attribute (PEP 495). For more details on migrating to a PEP 495-compliant implementation, see https://pytz-deprecation-shim.readthedocs.io/en/latest/migration.html
  date_obj = stz.localize(date_obj)


The reference epoch needs to be added as such to the analysis:

In [7]:
analysis.reference_epoch = reference_epoch

[2022-06-16 14:43:23][INFO] Building KDTree structure with leaf parameter 10
[2022-06-16 14:43:23][INFO] Saving epoch to file 'C:\Users\k53\AppData\Local\Temp\tmpgdw0m15r\reference_epoch.zip'


We also need to define the set of core points in use. In this example, we use all points in the reference epoch as core points:

In [8]:
analysis.corepoints = reference_epoch

[2022-06-16 14:43:26][INFO] Saving epoch to file 'C:\Users\k53\AppData\Local\Temp\tmpnl7jugt1\corepoints.zip'


Next, we want to add epochs to the spatiotemporal analysis. For this, we need to define a method that calculates the point cloud distances, for example using the M3C2 algorithm ([Lague et al., 2013](#references)). For details of the algorithm setup, you can have a look at the [M3C2 notebook](m3c2.ipynb).

In [9]:
analysis.m3c2 = py4dgeo.M3C2(cyl_radii=[2.0], normal_radii=[2.0])

Having done all of this, we can add an epoch to the analysis:

In [10]:
analysis.add_epochs(epoch)

[2022-06-16 14:43:32][INFO] Removing intermediate results from the analysis file H:\projects_hot\202206_py4dgeo\py4dgeo\jupyter\test.zip
[2022-06-16 14:43:32][INFO] Restoring epoch from file 'C:\Users\k53\AppData\Local\Temp\tmpu6n6dh2g\reference_epoch.zip'
[2022-06-16 14:43:32][INFO] Starting: Adding epoch 1/1 to analysis object
[2022-06-16 14:43:32][INFO] Building KDTree structure with leaf parameter 10
[2022-06-16 14:43:32][INFO] Finished in 0.0040s: Adding epoch 1/1 to analysis object
[2022-06-16 14:43:32][INFO] Starting: Rearranging space-time array in memory
[2022-06-16 14:43:32][INFO] Finished in 0.0023s: Rearranging space-time array in memory
[2022-06-16 14:43:32][INFO] Starting: Updating disk-based analysis archive with new epochs
[2022-06-16 14:43:32][INFO] Finished in 0.0118s: Updating disk-based analysis archive with new epochs


Having done this, we can inspect some of the data contained in the analysis:

In [11]:
np.set_printoptions(threshold=5)

In [12]:
print(f"Space-time distance array: {analysis.distances}")

Space-time distance array: [[-0.10153012]
 [-0.10094268]
 [-0.10040015]
 ...
 [-0.10110281]
 [-0.10172241]
 [-0.10335557]]


In [13]:
print(f"Uncertainty array of M3C2 calculation: {analysis.uncertainties}")

Uncertainty array of M3C2 calculation: [[(0.00218136, 0.        , 6, 0.00248861, 5)]
 [(0.00262324, 0.        , 8, 0.00354104, 7)]
 [(0.0025705 , 0.        , 9, 0.00370943, 8)]
 ...
 [(0.00396891, 0.00393397, 8, 0.00389378, 7)]
 [(0.00582156, 0.00629554, 8, 0.00520329, 7)]
 [(0.01303292, 0.01264785, 6, 0.00936853, 5)]]


In [14]:
print(f"Timestamp deltas of analysis: {analysis.timedeltas}")

Timestamp deltas of analysis: [datetime.timedelta(days=1)]


Note that the `add_epochs` can be passed several epochs at the same time to save some costly memory operations. Also, `add_epochs` can be called again on existing analysis objects allowing you to update your analysis after additional data acquisition at the same site.

<a id='references'></a>
## References

* Anders, K., Winiwarter, L., Lindenbergh, R., Williams, J. G., Vos, S. E., & Höfle, B. (2020). 4D objects-by-change: Spatiotemporal segmentation of geomorphic surface change from LiDAR time series. ISPRS Journal of Photogrammetry and Remote Sensing, 159, pp. 352-363. doi: [10.1016/j.isprsjprs.2019.11.025](https://doi.org/10.1016/j.isprsjprs.2019.11.025).
* Anders, K., Winiwarter, L., Mara, H., Lindenbergh, R., Vos, S. E., & Höfle, B. (2021). Fully automatic spatiotemporal segmentation of 3D LiDAR time series for the extraction of natural surface changes. ISPRS Journal of Photogrammetry and Remote Sensing, 173, pp. 297-308. doi: [10.1016/j.isprsjprs.2021.01.015](https://doi.org/10.1016/j.isprsjprs.2021.01.015).
* Lague, D., Brodu, N., & Leroux, J. (2013). Accurate 3D comparison of complex topography with terrestrial laser scanner: Application to the Rangitikei canyon (N-Z). ISPRS Journal of Photogrammetry and Remote Sensing, 82, pp. 10-26. doi: [10.1016/j.isprsjprs.2013.04.009](https://doi.org/10.1016/j.isprsjprs.2013.04.009).